# KubeFlow Pipeline: Github Issue Summarization using Tensor2Tensor

Currently, this notebook must be run from the Kubeflow JupyterHub installation, as described in the codelab.

In this notebook, we will show how to:

* Interactively define a KubeFlow Pipeline using the Pipelines Python SDK
* Submit and run the pipeline
* Add a step in the pipeline

This example pipeline trains a [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor/) model on Github issue data, learning to predict issue titles from issue bodies. It then exports the trained model and deploys the exported model to [Tensorflow Serving](https://github.com/tensorflow/serving). 
The final step in the pipeline launches a web app which interacts with the TF-Serving instance in order to get model predictions.

## Setup

Before any experiment can be conducted. We need to setup and initialize an environment: ensure all Python modules has been setup and configured, as well as python modules

Setting up python modules

In [1]:
!pip3 install --upgrade 'https://storage.googleapis.com/ml-pipeline/release/0.1.8/kfp.tar.gz' > /dev/null
!pip3 install --upgrade './extensions' > /dev/null
%load_ext extensions

You are using pip version 9.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


All imports goes here

In [2]:
import boto3
import kfp
from kfp import compiler
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.notebook

from ipython_secrets import get_secret
from kfp.compiler import Compiler

import extensions
import extensions.kaniko as kaniko

sess = boto3.session.Session()

Do some imports and set some variables.  Set the `WORKING_DIR` to a path under the Cloud Storage bucket you created earlier.

In [2]:
EXPERIMENT_NAME = 'Github issue summarization'
PROJECT_NAME = 'github-issues'

AWS_S3_BUCKET = get_secret('AWS_S3_BUCKET')
WORKING_DIR = f"s3://{AWS_S3_BUCKET}/${PROJECT_NAME}/"
GITHUB_TOKEN = get_secret('GITHUB_TOKEN')

DEPLOY_WEBAPP = 'false'

AWS_S3_BUCKET = get_secret('AWS_S3_BUCKET')

DOCKER_REGISTRY = get_secret('DOCKER_REGISTRY')
DOCKER_REGISTRY_SECRET = get_secret('DOCKER_REGISTRY_SECRET')

T2TTRAIN_IMAGE = f"{DOCKER_REGISTRY}/library/t2ttrain:latest"

The Kubeflow Pipeline (KFP) system requires an "Experiment" to group pipeline runs. 

To get reference to experiment we try naive but idempotent method. If experiment with desired name does not exists then retrieval function will `get_experiment()` with raise `ValueError`. In this case we will create a new KFP experiment. 

In [ ]:
client = kfp.Client()
try:
    exp = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    exp = client.create_experiment(EXPERIMENT_NAME)

## Compile Images

Before we can run training, we will build and compile docker container that we will use later in our pipeline

In [3]:
%%template Dockerfile.t2ttrain
FROM tensorflow/tensorflow:latest-gpu
RUN apt-get update -y
RUN apt-get install --no-install-recommends -y -q ca-certificates python-dev python-setuptools wget unzip git
RUN easy_install pip
RUN pip install tensor2tensor
RUN pip install tensorflow_hub
RUN pip install pyyaml==3.12 six==1.11.0
COPY src /ml
ENTRYPOINT ["python", "/ml/train_model.py"]

UsageError: Cell magic `%%template` not found.


In [4]:
@dsl.pipeline(
  name='Build',
  description='Kaniko docker ubild operations'
)
def build_images():
    s3_loc=f"s3://{AWS_S3_BUCKET}/{EXPERIMENT_NAME}/dockerbuild.tar.gz"
    kaniko.upload_build_context_to_s3(s3_loc)
    kaniko.aws_to_kube_secret(secret_name='awscreds', session=sess)
    
    op1 = dsl.ContainerOp(
        name='t2ttrain',
        image='gcr.io/kaniko-project/executor:latest',
        arguments=['--package', s3_loc,
                   '--dockerfile', 'Dockerfile.t2ttrain',
                   '--context', T2TTRAIN_IMAGE]
    ).apply(
        kaniko.use_aws_credentials(secret_name='awscreds')
    ).apply(
        kaniko.use_pull_secret(secret_name=DOCKER_REGISTRY_SECRET)
    )
    
#     op1 = KanikoOp(
#         name='t2ttrain',
#         dockerfile='Dockerfile.t2ttrain',
#         destination=T2TTRAIN_IMAGE,
#         package=f"s3://{AWS_S3_BUCKET}/{EXPERIMENT_NAME}/dockerbuild.tar.gz"
#     ).apply(
#         k.use_pull_secret(secret_name=DOCKER_REGISTRY_SECRET)
#     )
#     .apply(
#         secret_name='kaniko-awscreds',
#         session=boto3.session.Session()
#     )
    
#     .add_build_package(
#         package=f"s3://{AWS_S3_BUCKET}/{EXPERIMENT_NAME}/dockerbuild.tar.gz"
#     )

Compiler().compile(build_images, 'kaniko.tar.gz')

#     op2 = op1.copy()
#     op2.name = 'tfserve'
#     op2.destination=f"{DOCKER_REGISTRY}/library/tfserve:{DOCKER_TAG}",
#     op2.dockerfile='Dockerfile.tfserve'
    
#     op3 = op1.copy()
#     op3.name = 'app'
#     op3.destination=f"{DOCKER_REGISTRY}/library/app:{DOCKER_TAG}",
#     op3.dockerfile='Dockerfile.app'

In [ ]:
Compiler().compile(kaniko, 'kaniko.tar.gz')
r = client.run_pipeline(experiment.id, f'build {DOCKER_IMAGE}', 'kaniko.tar.gz')

In [ ]:
# block till completion
client.wait_for_run_completion(r.id, timeout=400).run.status

## Define a Pipeline

Authoring a pipeline is like authoring a normal Python function. The pipeline function describes the topology of the pipeline. 

Each step in the pipeline is typically a `ContainerOp` --- a simple class or function describing how to interact with a docker container image. In the pipeline, all the container images referenced in the pipeline are already built. 

The pipeline starts by training a [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor/) model, using already-preprocessed data. (More accurately, this step starts from an existing model checkpoint, then trains for a few more hundred steps).  When it finishes, it exports the model in a form suitable for serving by [TensorFlow serving](https://github.com/tensorflow/serving/).

The next step deploys a TF-serving instance with that model.

The last step launches a web app with which you can interact with the TF-serving instance to get model predictions.

In [ ]:
@dsl.pipeline(
    name='Github issue summarization',
    description='Tensor2Tensor-based training and TF-Serving'
)
def gh_summ(
  train_steps: dsl.PipelineParam=dsl.PipelineParam(name='train-steps', value=2019300),
  project: dsl.PipelineParam=dsl.PipelineParam(name='project', value='YOUR_PROJECT_HERE'),
  github_token: dsl.PipelineParam=dsl.PipelineParam(name='github-token', value='YOUR_GITHUB_TOKEN_HERE'),
  working_dir: dsl.PipelineParam=dsl.PipelineParam(name='working-dir', value='YOUR_GCS_DIR_HERE'),
  checkpoint_dir: dsl.PipelineParam=dsl.PipelineParam(name='checkpoint-dir', value='gs://aju-dev-demos-codelabs/kubecon/model_output_tbase.bak2019000'),
  deploy_webapp: dsl.PipelineParam=dsl.PipelineParam(name='deploy-webapp', value='true'),
  data_dir: dsl.PipelineParam=dsl.PipelineParam(name='data-dir', value='gs://aju-dev-demos-codelabs/kubecon/t2t_data_gh_all/')):


  train = dsl.ContainerOp(
      name = 'train',
      image = T2TTRAIN_IMAGE,
      arguments = [ "--data-dir", data_dir,
          "--checkpoint-dir", checkpoint_dir,
          "--model-dir", '%s/%s/model_output' % (working_dir, '{{workflow.name}}'),
          "--train-steps", train_steps, "--deploy-webapp" , deploy_webapp],
      file_outputs={'output': '/tmp/output'}

      ).apply(gcp.use_gcp_secret('user-gcp-sa'))

  serve = dsl.ContainerOp(
      name = 'serve',
      image = 'gcr.io/google-samples/ml-pipeline-kubeflow-tfserve',
      arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
          "--model_path", '%s/%s/model_output/export' % (working_dir, '{{workflow.name}}')
          ]
      )
  serve.after(train)
  train.set_gpu_limit(4)

  with dsl.Condition(train.output=='true'):
    webapp = dsl.ContainerOp(
        name = 'webapp',
        image = 'gcr.io/google-samples/ml-pipeline-webapp-launcher',
        arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
            "--github_token", github_token]

        )
    webapp.after(serve)

## Submit an experiment *run*

In [ ]:
compiler.Compiler().compile(gh_summ, 'ghsumm.tar.gz')

The call below will run the compiled pipeline.  We won't actually do that now, but instead we'll add a new step to the pipeline, then run it.

In [ ]:
# You'd uncomment this call to actually run the pipeline. 
# run = client.run_pipeline(exp.id, 'ghsumm', 'ghsumm.tar.gz',
#                           params={'working-dir': WORKING_DIR,
#                                   'github-token': GITHUB_TOKEN,
#                                   'project': PROJECT_NAME})

## Add a step to the pipeline

Next, let's add a new step to the pipeline above.  As currently defined, the pipeline accesses a directory of pre-processed data as input to training.  Let's see how we could include the pre-processing as part of the pipeline. 

We're going to cheat a bit, as processing the full dataset will take too long for this workshop, so we'll use a smaller sample. For that reason, you won't actually make use of the generated data from this step (we'll stick to using the full dataset for training), but this shows how you could do so if we had more time.

First, we'll define the new pipeline step. Note the last line of this new function, which gives this step's pod the credentials to access GCP.

In [ ]:
# defining the new data preprocessing pipeline step. 
# Note the last line, which gives this step's pod the credentials to access GCP
def preproc_op(data_dir, project):
  return dsl.ContainerOp(
    name='datagen',
    image='gcr.io/google-samples/ml-pipeline-t2tproc',
    arguments=[ "--data-dir", data_dir, "--project", project]
  ).apply(gcp.use_gcp_secret('user-gcp-sa'))

### Modify the pipeline to add the new step

Now, we'll redefine the pipeline to add the new step.

In [ ]:
# Then define a new Pipeline. It's almost the same as the original one, 
# but with the addition of the data processing step.
import kfp.dsl as dsl
import kfp.gcp as gcp

@dsl.pipeline(
  name='Github issue summarization',
  description='TFT-based feature processing, TFMA, TFJob, CMLE OP, and TF-Serving'
)
def gh_summ2(
  train_steps: dsl.PipelineParam=dsl.PipelineParam(name='train-steps', value=2019300),
  project: dsl.PipelineParam=dsl.PipelineParam(name='project', value='YOUR_PROJECT_HERE'),
  github_token: dsl.PipelineParam=dsl.PipelineParam(name='github-token', value='YOUR_GITHUB_TOKEN_HERE'),
  working_dir: dsl.PipelineParam=dsl.PipelineParam(name='working-dir', value='YOUR_GCS_DIR_HERE'),
  checkpoint_dir: dsl.PipelineParam=dsl.PipelineParam(name='checkpoint-dir', value='gs://aju-dev-demos-codelabs/kubecon/model_output_tbase.bak2019000'),
  deploy_webapp: dsl.PipelineParam=dsl.PipelineParam(name='deploy-webapp', value='true'),
  data_dir: dsl.PipelineParam=dsl.PipelineParam(name='data-dir', value='gs://aju-dev-demos-codelabs/kubecon/t2t_data_gh_all/')):

  # The new pre-processing op.
  preproc = preproc_op(project=project,
      data_dir=('%s/%s/gh_data' % (working_dir, '{{workflow.name}}')))

  train = dsl.ContainerOp(
      name = 'train',
      image = 'gcr.io/google-samples/ml-pipeline-t2ttrain',
      arguments = [ "--data-dir", data_dir,
          "--checkpoint-dir", checkpoint_dir,
          "--model-dir", '%s/%s/model_output' % (working_dir, '{{workflow.name}}'),
          "--train-steps", train_steps, "--deploy-webapp" , deploy_webapp],
      file_outputs={'output': '/tmp/output'}

      ).apply(gcp.use_gcp_secret('user-gcp-sa'))
  train.after(preproc)

  serve = dsl.ContainerOp(
      name = 'serve',
      image = 'gcr.io/google-samples/ml-pipeline-kubeflow-tfserve',
      arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
          "--model_path", '%s/%s/model_output/export' % (working_dir, '{{workflow.name}}')
          ]
      )
  serve.after(train)
  train.set_gpu_limit(4)

  with dsl.Condition(train.output=='true'):
    webapp = dsl.ContainerOp(
        name = 'webapp',
        image = 'gcr.io/google-samples/ml-pipeline-webapp-launcher',
        arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
            "--github_token", github_token]

        )
    webapp.after(serve)   


### Compile the new pipeline definition and submit the run

In [ ]:
compiler.Compiler().compile(gh_summ2, 'ghsumm2.tar.gz')

In [ ]:
run = client.run_pipeline(exp.id, 'ghsumm2', 'ghsumm2.tar.gz',
                          params={'working-dir': WORKING_DIR,
                                  'github-token': GITHUB_TOKEN,
                                  'deploy-webapp': DEPLOY_WEBAPP,
                                  'project': PROJECT_NAME})

![The new pipeline.](https://storage.googleapis.com/amy-jo/images/datagen_t2t_pipeline.png)

When this new pipeline finishes running, you'll be able to see your generated processed data files in GCS under the path: `WORKING_DIR/<pipeline_name>/gh_data`. There isn't time in the workshop to pre-process the full dataset, but if there had been, we could have defined our pipeline to read from that generated directory for its training input.

-----------------------------
Copyright 2018 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.